In [1]:
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import torch.utils.data as Data
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
import time
import multiprocessing as mp
from os import walk
day = 30
stride = 10

In [2]:
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(data_length, stride):
    if data_length % stride == 0:
        step_list = list(range(0, data_length + stride, stride))
    elif data_length % stride <= 5:
        mod = data_length % stride
        step_list = list(range(0, data_length - stride, stride)) + [data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
    return step_list

In [3]:
path_list = []
data_path = "/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/"
output = "/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list

100%|██████████| 26/26 [00:00<00:00, 364722.09it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2016-01-01_2016-03-01.par',
 '2017-01-01_2017-03-01.par',
 '2015-06-01_2015-09-01.par',
 '2018-01-01_2018-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2021-01-01_2021-03-01.par',
 '2016-06-01_2016-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2019-01-01_2019-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2021-03-01_2021-06-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-06-01_2017-09-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-09-01_2021-01-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-03-01_2019-06-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-09-01_2020-01-01.par']

In [4]:
# Read Data
def test(input_path,name,output_path):
    print("read" + input_path + name)
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(data_length, stride)

    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov") + name)
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [5]:
pool = mp.Pool(10)
t1 = time.time()
for i in path_list:
#     test(data_path,i,output)
    pool.apply_async(test, args=(data_path,i,output))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-03-01_2015-06-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-01-01_2015-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-01-01_2016-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2017-01-01_2017-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-03-01_2016-06-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-06-01_2015-09-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2021-01-01_2021-03-01.par
read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2016-06-01_2016-09-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2018-01-01_2018-03-01.parread/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-09-01_2016-01-01.par










  0%|          | 0/3 [00:00<?, ?it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2015-01-01_2015-03-01.par
read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2020-01-01_2020-03-01.par


100%|██████████| 3/3 [00:10<00:00,  3.49s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2016-01-01_2016-03-01.par
read/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2019-01-01_2019-03-01.par


...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2019-03-01_2019-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2020-09-01_2021-01-01.par


100%|██████████| 3/3 [00:23<00:00,  7.76s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2017-09-01_2018-01-01.par


100%|██████████| 3/3 [00:25<00:00,  8.36s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2018-09-01_2019-01-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Corr/2019-09-01_2020-01-01.par
并行执行时间： 83


(118, 55, 3)

In [2]:
dataframe_list = pd.read_parquet('/home/wuwenjun/Data/AlphaNet_Original_Input/2020-12-31_2021-06-30.par')
dataframe_list = dataframe_list[dataframe_list['ticker'] == 1]
dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
dataframe_list

,,2,3,4,5,6,7,8,9,10,11,...,323,324,325,326,327,328,329,330,331,target
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2020-12-31 09:30:00,1,18.46,18.59,18.98,18.30,18.85,18.7373,1211740.62,2.1127,0.6244,1.4088,...,19.20,18.72,19.20,18.9483,978497.78,0.1565,0.5042,1.1376,0.03,0.035919
2021-01-04 09:30:00,1,18.85,18.83,18.99,18.52,18.86,18.7747,673919.22,0.0531,0.3473,0.7835,...,19.58,19.02,19.34,19.2724,924503.43,0.7292,0.4764,1.0749,0.14,0.047120
2021-01-05 09:30:00,1,18.86,18.85,19.62,18.80,19.62,19.3187,1165858.26,4.0297,0.6008,1.3555,...,19.10,18.44,18.60,18.6054,1554216.43,-3.8263,0.8009,1.8070,-0.74,0.108152
2021-01-06 09:30:00,1,19.62,19.62,19.68,19.17,19.36,19.3576,678543.23,-1.3252,0.3497,0.7889,...,18.48,17.80,18.17,18.0339,1821352.10,-2.3118,0.9386,2.1176,-0.43,0.161504
2021-01-07 09:30:00,1,19.36,19.48,19.70,19.05,19.06,19.3337,552585.01,-1.5496,0.2848,0.6425,...,19.56,18.00,19.56,18.8559,1934945.12,7.6500,0.9971,2.2496,1.39,0.059426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 09:30:00,1,23.86,23.62,23.75,23.06,23.53,23.3667,497223.45,-1.3831,0.2562,0.5781,...,23.09,22.25,22.97,22.6995,757751.41,3.6552,0.3905,0.8810,0.81,-0.021444
2021-06-24 09:30:00,1,23.53,23.29,23.64,22.88,23.55,23.3564,533265.26,0.0850,0.2748,0.6200,...,23.41,22.72,23.14,23.1261,447907.78,0.7401,0.2308,0.5208,0.17,-0.009553
2021-06-25 09:30:00,1,23.55,23.52,23.59,22.84,23.07,23.1409,549652.50,-2.0382,0.2832,0.6390,...,23.12,22.68,23.08,22.9193,390283.49,-0.2593,0.2011,0.4538,-0.06,-0.015598


In [5]:
x = np.array(dataframe_list.drop('target',axis=1))
x = x.reshape(x.shape[0], 1, day, -1)
x = x.transpose(0, 1, 3, 2)

feat_nums = x.shape[2]
data_length = x.shape[3]
num, num_rev = generate_Num_and_ReversedNum(feat_nums)
step_list = generate_Step_List(data_length, stride)

In [11]:
x.shape

(118, 1, 11, 30)

In [56]:
check_data = pd.read_csv("/home/wuwenjun/Data/AlphaNet_Original_Input.csv")
check_data = check_data[check_data['ticker'] == 1]
check_data['timestamp'] = pd.to_datetime(check_data['timestamp'])
check_data = check_data[(check_data['timestamp'] > pd.to_datetime("20201120")) & (check_data['timestamp'] < pd.to_datetime("20210101"))]
check_data = check_data.set_index(['timestamp','ticker']).drop('5d_ret',axis=1).T
check_data.shape

(11, 30)

In [69]:
l = []
for i in tqdm(range(len(step_list) - 1)):
    start = step_list[i]
    end = step_list[i + 1]
    sub_data1 = x[:, :, num, start:end]
    sub_data2 = x[:, :, num_rev, start:end]
    std1 = sub_data1.std(axis=4, keepdims=True)
    std2 = sub_data2.std(axis=4, keepdims=True)
    std = (std1 * std2).mean(axis=3, keepdims=True)
    l.append(std)
std = np.squeeze(np.array(l)).transpose(1, 2, 0)
fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
l = np.divide(cov, std, out=np.zeros_like(cov, dtype=np.float64), where=std!=0)* fct
l.shape

100%|██████████| 3/3 [00:00<00:00, 274.97it/s]


(118, 55, 3)

(118, 55, 3)

In [35]:
cov = pd.read_parquet('/home/wuwenjun/Data/AlphaNet_Original_Input_Corr/2020-12-31_2021-06-30.par')
cov = cov[cov['ticker'] == 1]
cov = np.array(cov.set_index(['timestamp','ticker']))
cov = cov.reshape(std.shape)

(118, 55, 3)

In [36]:
cov[0]

array([[ 2.43177778e-01,  1.50045556e-01,  1.92415556e-01],
       [ 2.34722222e-01,  1.13832222e-01,  1.62180000e-01],
       [ 2.00555556e-01,  9.40922222e-02,  1.50908889e-01],
       [ 1.33122222e-01,  9.98744444e-02,  1.55820000e-01],
       [ 1.96336111e-01,  9.32777667e-02,  1.49562267e-01],
       [ 8.98467423e+03,  9.49268868e+03,  1.50811459e+04],
       [-5.95909222e-01, -2.74833411e-01, -3.50752400e-01],
       [ 4.62888889e-03,  4.89352222e-03,  7.77237778e-03],
       [ 1.04456667e-02,  1.10361333e-02,  1.75297111e-02],
       [-1.13255556e-01, -5.26688889e-02, -6.39533333e-02],
       [ 2.51994444e-01,  1.21056667e-01,  1.44008889e-01],
       [ 2.11544444e-01,  1.01796667e-01,  1.31535556e-01],
       [ 1.36672222e-01,  1.04536667e-01,  1.26330000e-01],
       [ 2.08711500e-01,  1.02386011e-01,  1.29162122e-01],
       [ 1.14742537e+04,  7.21178113e+03,  1.18384210e+04],
       [-5.63552556e-01, -2.37064700e-01, -3.61613156e-01],
       [ 5.90988889e-03,  3.71925556e-03

In [58]:
check_data

timestamp,2020-11-20 09:30:00,2020-11-23 09:30:00,2020-11-24 09:30:00,2020-11-25 09:30:00,2020-11-26 09:30:00,2020-11-27 09:30:00,2020-11-30 09:30:00,2020-12-01 09:30:00,2020-12-02 09:30:00,2020-12-03 09:30:00,...,2020-12-18 09:30:00,2020-12-21 09:30:00,2020-12-22 09:30:00,2020-12-23 09:30:00,2020-12-24 09:30:00,2020-12-25 09:30:00,2020-12-28 09:30:00,2020-12-29 09:30:00,2020-12-30 09:30:00,2020-12-31 09:30:00
ticker,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
pre_close,1.846000e+01,18.8500,1.886000e+01,19.6200,19.3600,19.0600,19.5000,1.970000e+01,1.974000e+01,20.0500,...,19.0100,1.895000e+01,18.3600,1.828000e+01,17.9000,18.0000,18.2600,1.804000e+01,18.8500,19.1700
open,1.859000e+01,18.8300,1.885000e+01,19.6200,19.4800,19.0500,20.0000,1.990000e+01,1.970000e+01,19.9300,...,18.9100,1.900000e+01,18.3000,1.836000e+01,17.9600,18.1500,18.2600,1.802000e+01,18.8700,19.0000
high,1.898000e+01,18.9900,1.962000e+01,19.6800,19.7000,19.6100,20.0000,2.088000e+01,2.051000e+01,20.0600,...,19.0700,1.910000e+01,18.6400,1.865000e+01,18.0700,18.4900,18.2600,1.886000e+01,19.3000,19.2000
low,1.830000e+01,18.5200,1.880000e+01,19.1700,19.0500,19.0300,19.3800,1.959000e+01,1.940000e+01,19.5200,...,18.6000,1.823000e+01,18.0300,1.788000e+01,17.7900,18.0500,17.8000,1.796000e+01,18.7000,18.7200
close,1.885000e+01,18.8600,1.962000e+01,19.3600,19.0600,19.5000,19.7000,1.974000e+01,2.005000e+01,19.6300,...,18.9500,1.836000e+01,18.2800,1.790000e+01,18.0000,18.2600,18.0400,1.885000e+01,19.1700,19.2000
vwap,1.873730e+01,18.7747,1.931870e+01,19.3576,19.3337,19.3865,19.6270,2.032120e+01,1.990960e+01,19.7736,...,18.8687,1.851010e+01,18.3338,1.821300e+01,17.9400,18.2600,17.9894,1.852220e+01,19.0838,18.9483
volume,1.211741e+06,673919.2200,1.165858e+06,678543.2300,552585.0100,639657.8900,753773.7400,1.581441e+06,1.263720e+06,889385.2900,...,686140.6900,1.058756e+06,906878.1100,1.028738e+06,750082.7300,632950.1200,577077.3300,1.270337e+06,963092.2300,978497.7800
pct_chg,2.112700e+00,0.0531,4.029700e+00,-1.3252,-1.5496,2.3085,1.0256,2.030000e-01,1.570400e+00,-2.0948,...,-0.3156,-3.113500e+00,-0.4357,-2.078800e+00,0.5587,1.4444,-1.2048,4.490000e+00,1.6976,0.1565
turnover,6.244000e-01,0.3473,6.008000e-01,0.3497,0.2848,0.3296,0.3884,8.149000e-01,6.512000e-01,0.4583,...,0.3536,5.456000e-01,0.4673,5.301000e-01,0.3865,0.3262,0.2974,6.546000e-01,0.4963,0.5042


In [61]:

test_list = []
k = 0
for i in range(0,10):
    for j in range(i+1,11):
        for time in range(len(step_list) - 1):
            k += 1
            start = step_list[time]
            end = step_list[time + 1]
            test_list.append(check_data.iloc[[i,j],start:end].T.corr().iloc[0,1])
len(test_list)

165

In [62]:
test_list = pd.DataFrame(np.array(test_list).reshape(55,3))
test_list

,0,1,2
0,0.940634,0.983912,0.984650
1,0.793122,0.881220,0.838432
2,0.941421,0.779884,0.876099
3,0.671183,0.814636,0.670585
4,0.816260,0.771008,0.806222
5,0.052799,0.241532,0.147047
6,-0.616521,-0.587132,-0.350213
7,0.052792,0.241601,0.147083
8,0.052799,0.241516,0.147010
9,-0.610244,-0.595219,-0.347843


In [111]:
check_data = pd.read_parquet('/home/wuwenjun/Data/AlphaNet_Original_Input_Corr/2020-12-31_2021-06-30.par')
check_data = check_data[check_data['ticker'] == 1]
check_data = check_data.set_index(['ticker',"timestamp"])
check_data

0         1         2         3         4  \
ticker timestamp                                                               
1      2020-12-31 09:30:00  0.940634  0.983912  0.984650  0.793122  0.881220   
       2021-01-04 09:30:00  0.917992  0.982130  0.986525  0.703494  0.807413   
       2021-01-05 09:30:00  0.878411  0.961364  0.985041  0.519465  0.653137   
       2021-01-06 09:30:00  0.805523  0.942429  0.978538  0.551010  0.594230   
       2021-01-07 09:30:00  0.876860  0.954973  0.978968  0.709478  0.707179   
...                              ...       ...       ...       ...       ...   
       2021-06-23 09:30:00  0.798385  0.972618  0.988442  0.690358  0.404802   
       2021-06-24 09:30:00  0.808385  0.956779  0.987757  0.456169  0.567104   
       2021-06-25 09:30:00  0.941532  0.960344  0.984030  0.763830  0.557077   
       2021-06-28 09:30:00  0.968544  0.943178  0.969857  0.853930  0.455532   
       2021-06-29 09:30:00  0.976099  0.912830  0.952602  0.808067  0.443744   

                                   5         6         7         8         9  \
ticker timestamp                                                               
1      2020-12-31 09:30:00  0.838432  0.941421  0.779884  0.876099  0.671183   
       2021-01-04 09:30:00  0.874288  0.893427  0.641992  0.885741  0.523315   
       2021-01-05 09:30:00  0.838380  0.805139  0.220056  0.870556  0.255812   
       2021-01-06 09:30:00  0.808278  0.714332  0.323796  0.816867  0.515001   
       2021-01-07 09:30:00  0.598222  0.820973  0.531289  0.816448  0.698405   
...                              ...       ...       ...       ...       ...   
       2021-06-23 09:30:00  0.950290  0.682771  0.826423  0.923575  0.275608   
       2021-06-24 09:30:00  0.945125  0.659882  0.802226  0.915252  0.130666   
       2021-06-25 09:30:00  0.917276  0.887558  0.830945  0.891069  0.621211   
       2021-06-28 09:30:00  0.826413  0.941938  0.767720  0.734674  0.748053   
       2021-06-29 09:30:00  0.609212  0.925705  0.579427  0.627047  0.715692   

                            ...       155  156  157  158       159       160  \
ticker timestamp            ...                                                
1      2020-12-31 09:30:00  ...  0.999796  1.0  1.0  1.0  0.377738 -0.300689   
       2021-01-04 09:30:00  ...  0.999769  1.0  1.0  1.0  0.348936 -0.302528   
       2021-01-05 09:30:00  ...  0.999557  1.0  1.0  1.0  0.339511 -0.565505   
       2021-01-06 09:30:00  ...  0.999608  1.0  1.0  1.0  0.209239 -0.509690   
       2021-01-07 09:30:00  ...  0.999760  1.0  1.0  1.0  0.192394 -0.636980   
...                         ...       ...  ...  ...  ...       ...       ...   
       2021-06-23 09:30:00  ...  0.999192  1.0  1.0  1.0 -0.149041  0.860046   
       2021-06-24 09:30:00  ...  0.999262  1.0  1.0  1.0  0.639480  0.734533   
       2021-06-25 09:30:00  ...  0.999289  1.0  1.0  1.0  0.621370  0.656702   
       2021-06-28 09:30:00  ...  0.999408  1.0  1.0  1.0  0.684083  0.604372   
       2021-06-29 09:30:00  ...  0.999817  1.0  1.0  1.0  0.688069 -0.049500   

                                 161       162       163       164  
ticker timestamp                                                    
1      2020-12-31 09:30:00  0.235218  0.377795 -0.300765  0.235123  
       2021-01-04 09:30:00  0.226293  0.348995 -0.302591  0.226198  
       2021-01-05 09:30:00 -0.213902  0.339565 -0.565552 -0.213935  
       2021-01-06 09:30:00 -0.378211  0.209362 -0.509762 -0.378222  
       2021-01-07 09:30:00  0.178542  0.192495 -0.637000  0.178535  
...                              ...       ...       ...       ...  
       2021-06-23 09:30:00 -0.235670 -0.149350  0.859964 -0.235808  
       2021-06-24 09:30:00 -0.280873  0.639354  0.734387 -0.280954  
       2021-06-25 09:30:00 -0.245116  0.621249  0.656450 -0.245174  
       2021-06-28 09:30:00 -0.287607  0.683999  0.604078 -0.287738  
       2021-06-29 09:30:00 -0.047164  0.687937 -0.049772 -0.047340  

[118 rows x 16

In [112]:
check_data = np.array(check_data).reshape(118,55,3)
check_data[0]

array([[ 0.94063353,  0.98391249,  0.98465013],
       [ 0.79312154,  0.8812197 ,  0.83843163],
       [ 0.94142099,  0.77988365,  0.87609921],
       [ 0.67118331,  0.81463624,  0.67058486],
       [ 0.81625953,  0.77100774,  0.80622167],
       [ 0.0527989 ,  0.24153239,  0.14704713],
       [-0.61652076, -0.58713231, -0.35021339],
       [ 0.05279201,  0.24160117,  0.14708306],
       [ 0.05279876,  0.24151613,  0.14701026],
       [-0.61024389, -0.59521876, -0.34784311],
       [ 0.811474  ,  0.93742011,  0.83729099],
       [ 0.94634373,  0.8439879 ,  0.85881257],
       [ 0.65670293,  0.85291267,  0.61143994],
       [ 0.82693727,  0.84654061,  0.78304074],
       [ 0.06426064,  0.18355036,  0.12981735],
       [-0.55564847, -0.50659385, -0.40606268],
       [ 0.06423458,  0.18367923,  0.12987046],
       [ 0.06426601,  0.18355139,  0.12979487],
       [-0.54690801, -0.51445247, -0.40424394],
       [ 0.91088168,  0.93651453,  0.87138195],
       [ 0.84693213,  0.93257047,  0.830

In [2]:
class Convolutional(object):
    def __init__(self, data, stride):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = np.array(data)
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2]  # 9
        self.num, self.num_rev = self.generate_Num_and_ReversedNum(self.feat_num)
        self.conv_feat = len(self.num)
        self.step_list = self.generate_Step_List(self.data_length, self.stride)
        self.extracted_data = self.Extraction(self.data, self.feat_num, self.conv_feat, self.stride)

    def Extraction(self, data, feat_num, conv_feat, stride):
        print("------Start Extraction------")
        batch = nn.BatchNorm1d(conv_feat, affine=True)
        batch2 = nn.BatchNorm1d(feat_num, affine=True)
        conv1 = self.ts_cov4d(self.data, self.stride, self.num, self.num_rev, self.step_list)
        conv2 = self.ts_corr4d(self.data, self.stride, self.num, self.num_rev, self.step_list, conv1)
        conv2 = torch.tanh(conv2)
        bc1 = batch(conv1.to(torch.float))
        bc2 = batch(conv2.to(torch.float))
        conv3 = self.ts_stddev4d(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
        bc3 = batch2(conv3)
        conv4 = self.ts_zscore(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
        bc4 = batch2(conv4)
        conv5 = self.ts_return(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
        bc5 = batch2(conv5)
        conv6 = self.ts_decaylinear(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
        bc6 = batch2(conv6)

        feat_cat = torch.cat([bc1, bc2, bc3, bc4, bc5, bc6], axis=1)  # ì??÷??o?
        shape = feat_cat.shape
        feat_cat = feat_cat.reshape(shape[0], 1, shape[1], shape[2])
        print("Convolutional shape: ", feat_cat.shape)
        return feat_cat

    def generateC(self, l1):
        if len(l1) == 1:
            return []
        v = [[l1[0], i] for i in l1[1:]]
        l1 = l1[1:]
        return v + self.generateC(l1)

    def generate_Num_and_ReversedNum(self, feat_nums):
        list1 = list(range(feat_nums))
        num = self.generateC(list1)
        num_rev = []
        for l in num:
            l1 = l.copy()
            l1.reverse()
            num_rev.append(l1)
        return num, num_rev

    def generate_Step_List(self, data_length, stride):
        # 11?¨2?3¤áD±í￡?è?1?êy?Y3¤?è2??ü??3y￡??òè?ê￡??3¤?è￡?è?1?ê￡??3¤?èD?óú5￡??òó?é?ò?2??áo?ò??e
        if data_length % stride == 0:
            step_list = list(range(0, data_length + stride, stride))
        elif data_length % stride <= 5:
            mod = data_length % stride
            step_list = list(range(0, data_length - stride, stride)) + [data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
        return step_list

    """ Main Extraction"""

    def ts_cov4d(self, data, stride, num, num_rev, step_list):
        '''????4??êy?Yμ?D-·?2?'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        # ????μ?1y3ì?D??±?±￡3?keepdims=True
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, num, start:end]  # (2000, 1, 36, 2, 10)
            sub_data2 = data[:, :, num_rev, start:end]
            mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
            mean2 = sub_data2.mean(axis=4, keepdims=True)
            spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
            spread2 = sub_data2 - mean2
            cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                   keepdims=True)  # (2000, 1, 36, 1, 1)
            l.append(cov)
        corr = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, self.conv_feat,
                                                                  len(step_list) - 1)  # (2000, 1, 36, 3)
        final = torch.from_numpy(corr)
        print("------Finished ts_cov4d----output shape: ", final.shape)
        return final

    def ts_corr4d(self, data, stride, num, num_rev, step_list, cov):
        '''????4??êy?Yμ??à1??μêy'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, num, start:end]
            sub_data2 = data[:, :, num_rev, start:end]
            std1 = sub_data1.std(axis=4, keepdims=True)
            std2 = sub_data2.std(axis=4, keepdims=True)
            std = (std1 * std2).mean(axis=3, keepdims=True)
            del std1, std2  # êí·??ú′?
            l.append(std)
        std = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, self.conv_feat, len(step_list) - 1)
        std[std == 0] = 1e-9
        fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
        final = cov / torch.from_numpy(std) * fct
        del fct, std
        print("------Finished ts_corr4d----output shape: ", final.shape)
        return final

    def ts_stddev4d(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, :, start:end]
            std1 = sub_data1.std(axis=3, keepdims=True)
            l.append(std1)
            del std1
        std = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        print("------Finished ts_stddev4d----output shape: ", torch.from_numpy(std).shape)
        return torch.from_numpy(std)

    def ts_zscore(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, :, start:end]
            mean = sub_data1.mean(axis=3, keepdims=True)
            std = sub_data1.std(axis=3, keepdims=True)
            std[std == 0] = 1e-9
            z_score = mean / std
            l.append(z_score)
        z_score = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        #         z_score[z_score >= 6] = 6
        print("------Finished ts_zscore----output shape: ", torch.from_numpy(z_score).shape)
        return torch.from_numpy(z_score)

    def ts_return(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        data[data == 0] = 1e-9
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, :, start:end]
            ret = sub_data1[:, :, :, -1] / sub_data1[:, :, :, 0] - 1
            l.append(ret)
        z_data = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        z_data[z_data > 1] = 1
        print("------Finished ts_return----output shape: ", torch.from_numpy(z_data).shape)
        return torch.from_numpy(z_data)

    def ts_decaylinear(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            time_spread = end - start
            weight = np.arange(1, time_spread + 1)
            weight = weight / (weight.sum())
            sub_data1 = (data[:, :, :, start:end] * weight).mean(axis=3, keepdims=True)
            l.append(sub_data1)
        decay_data = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        final = torch.from_numpy(decay_data)
        print("------Finished ts_decaylinear----output shape: ", final.shape)
        return final


class Pooling(object):
    def __init__(self, data, stride):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = data.detach().numpy()
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2]  # 9
        self.step_list = self.generate_Step_List(self.data_length, self.stride)
        self.extracted_data = self.Extraction(self.data, self.feat_num, self.stride)

    def Extraction(self, data, feat_num, stride):
        print("------Start Pooling------")
        # Pooling
        ts_max = self.ts_pool(data, self.stride, self.feat_num, self.step_list, method='max')
        ts_max = nn.BatchNorm1d(self.feat_num, affine=True)(ts_max)
        ts_min = self.ts_pool(data, self.stride, self.feat_num, self.step_list, method='min')
        ts_min = nn.BatchNorm1d(self.feat_num, affine=True)(ts_min)
        ts_mean = self.ts_pool(data, self.stride, self.feat_num, self.step_list, method='mean')
        ts_mean = nn.BatchNorm1d(self.feat_num, affine=True)(ts_mean)
        data_pool = torch.cat([ts_max, ts_min, ts_mean], axis=1)
        data_pool = data_pool.flatten(start_dim=1)
        print("Pooling shape: ", data_pool.shape)
        return data_pool

    def generate_Step_List(self, data_length, stride):
        # 11?¨2?3¤áD±í￡?è?1?êy?Y3¤?è2??ü??3y￡??òè?ê￡??3¤?è￡?è?1?ê￡??3¤?èD?óú5￡??òó?é?ò?2??áo?ò??e
        if data_length % stride == 0:
            step_list = list(range(0, data_length + stride, stride))
        elif data_length % stride <= 5:
            mod = data_length % stride
            step_list = list(range(0, data_length - stride, stride)) + [data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
        return step_list

    def ts_pool(self, data, stride, feat_num, step_list, method):
        if type(data) == torch.Tensor:
            data = data.detach().numpy()
        if data.shape[-1] <= stride:
            step_list = [0, data.shape[-1]]
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            if method == 'max':
                sub_data1 = data[:, :, :, start:end].max(axis=3, keepdims=True)
            if method == 'min':
                sub_data1 = data[:, :, :, start:end].min(axis=3, keepdims=True)
            if method == 'mean':
                sub_data1 = data[:, :, :, start:end].mean(axis=3, keepdims=True)
            l.append(sub_data1)
        try:
            pool_data = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        except:
            pool_data = np.squeeze(np.array(l)).reshape(-1, feat_num, len(step_list) - 1)
        return torch.from_numpy(pool_data)


class AlphaNet(nn.Module):
    def __init__(self, factor_num, fully_connect_layer_neural):
        # super ??àà￡?μ÷ó???ààμ?11?ì￡??aò?2?±?D?óD
        # μúò???2?êy?a?¨ò?ààμ???3?￡?μú?t???aself
        super(AlphaNet, self).__init__()
        self.fc1_neuron = (factor_num * (factor_num - 1) + 4 * factor_num) * 3
        self.fc2_neuron = fully_connect_layer_neural
        self.fc1 = torch.nn.Linear(self.fc1_neuron, self.fc2_neuron)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.out = nn.Linear(self.fc2_neuron, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred


def get_train_data(time_start, time_end):
#     train_frame = dataframe_list[dataframe_list['timestamp'] < pd.to_datetime(str(time_start))]
    train_frame = dataframe_list[(dataframe_list['timestamp'] > pd.to_datetime(str(20200630)))
                                & (dataframe_list['timestamp'] < pd.to_datetime(str(time_start)))]
    train_frame.set_index(["timestamp", "ticker"], inplace=True)

    # Train X and Train Y
    trainx, trainy = [], []
    trainx = np.array(train_frame.drop("target", axis=1))
    trainx = trainx.reshape(trainx.shape[0], 1, day, -1)  # x = (153,1,30,9)
    trainx = trainx.transpose(0, 1, 3, 2)  # x = (2555577, 1, 9, 30)
    trainy = np.array(train_frame['target']).reshape(-1, 1)  # x = (153,1,9,30)

    feat_num = trainx.shape[2]  # 9
    del train_frame
    print("trainx.shape: ", trainx.shape)
    print("trainy.shape: ", trainy.shape)
    return trainx, trainy, feat_num


def get_test_data(time_start, time_end):
    test_frame = dataframe_list[(dataframe_list['timestamp'] > pd.to_datetime(str(time_start)))
                                & (dataframe_list['timestamp'] < pd.to_datetime(str(time_end)))]
    test_frame.set_index(["timestamp", "ticker"], inplace=True)

    # Test X and Test Y

    test_target = pd.DataFrame(test_frame['target'])
    testy = np.array(test_target).reshape(-1, 1)

    testx = np.array(test_frame.drop("target", axis=1))
    testx = testx.reshape(testx.shape[0], 1, day, -1)  # x = (153,1,30,9)
    testx = testx.transpose(0, 1, 3, 2)  # x = (2555577, 1, 9, 30)
    del test_frame

    print("testx.shape: ", testx.shape)
    print("testy.shape: ", testy.shape)
    test_target.reset_index(inplace=True)
    return testx, testy, test_target

output_path = "/home/wuwenjun/Alpha_Factor/AlphaNetV1_Original_Input_1208/"
day = 30
stride = 10

# Read Data
data_path = "/home/wuwenjun/Data/AlphaNet_Original_Input/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        dataframe_list = pd.concat([dataframe_list, pd.read_parquet(f + j)], axis=0)
dataframe_list['timestamp'] = pd.to_datetime(dataframe_list['timestamp'])

100%|██████████| 8/8 [01:21<00:00, 10.16s/it]


In [3]:
time_start = 20210101
time_end = 20210630
trainx, trainy, feat_num = get_train_data(time_start, time_end)
testx, testy, test_target = get_test_data(time_start, time_end)
del dataframe_list
"""Convolutional """
convolutional = Convolutional(trainx, 10)
feat_cat = convolutional.extracted_data
pooling = Pooling(feat_cat, 3)
trainx = pooling.extracted_data.detach().numpy()
print("trainx.shape : ", trainx.shape)
print("trainy.shape : ", trainy.shape)

convolutional = Convolutional(testx, 10)
feat_cat = convolutional.extracted_data
pooling = Pooling(feat_cat, 3)
testx = pooling.extracted_data.detach().numpy()
print("testx.shape : ", testx.shape)
print("testy.shape : ", testy.shape)

trainx, trainy, testx, testy = torch.from_numpy(trainx), torch.from_numpy(trainy), torch.from_numpy(
    testx), torch.from_numpy(testy)
print('trainx size: ', trainx.size())
print('trainy size: ', trainy.size())
print('testx size: ', testx.size())
print('testy size: ', testy.size())
torch.save(trainx, output_path + "/trainx/" + '%s_%s.pt' % (time_start, time_end))
torch.save(trainy, output_path + "/trainy/" + '%s_%s.pt' % (time_start, time_end))
torch.save(testx, output_path + "/testx/" + '%s_%s.pt' % (time_start, time_end))
torch.save(testy, output_path + "/testy/" + '%s_%s.pt' % (time_start, time_end))
test_target.set_index(["timestamp","ticker"]).to_csv(output_path + "/test_target/" + '%s_%s.csv' % (time_start, time_end))

trainx.shape:  (459371, 1, 11, 30)
trainy.shape:  (459371, 1)
testx.shape:  (451184, 1, 11, 30)
testy.shape:  (451184, 1)


  0%|          | 0/3 [00:00<?, ?it/s]

------Start Extraction------


  0%|          | 0/3 [00:00<?, ?it/s]

------Finished ts_cov4d----output shape:  torch.Size([459371, 55, 3])


...

------Finished ts_decaylinear----output shape:  torch.Size([451184, 11, 3])
Convolutional shape:  torch.Size([451184, 1, 154, 3])


  0%|          | 0/1 [00:00<?, ?it/s]

------Start Pooling------


100%|██████████| 1/1 [00:01<00:00,  1.21s/it]


Pooling shape:  torch.Size([451184, 462])
testx.shape :  (451184, 462)
testy.shape :  (451184, 1)
trainx size:  torch.Size([459371, 462])
trainy size:  torch.Size([459371, 1])
testx size:  torch.Size([451184, 462])
testy size:  torch.Size([451184, 1])


In [17]:
trainx = torch.load("/home/wuwenjun/Alpha_Factor/AlphaNetV1_Original_Input_1208/trainx/20190630_20200101.pt")
trainy = torch.load("/home/wuwenjun/Alpha_Factor/AlphaNetV1_Original_Input_1208/trainy/20190630_20200101.pt")
testx = torch.load("/home/wuwenjun/Alpha_Factor/AlphaNetV1_Original_Input_1208/testx/20190630_20200101.pt")
testy = torch.load("/home/wuwenjun/Alpha_Factor/AlphaNetV1_Original_Input_1208/testy/20190630_20200101.pt")
test_target = pd.read_csv("/home/wuwenjun/Alpha_Factor/AlphaNetV1_Original_Input_1208/test_target/20190630_20200101.csv")
print('trainx size: ', trainx.size())
print('trainy size: ', trainy.size())
print('testx size: ', testx.size())
print('testy size: ', testy.size())
test_target

trainx size:  torch.Size([2961314, 462])
trainy size:  torch.Size([2961314, 1])
testx size:  torch.Size([436426, 462])
testy size:  torch.Size([436426, 1])


,timestamp,ticker,target
0,2019-07-01 09:30:00,1,-0.013562
1,2019-07-02 09:30:00,1,-0.018012
2,2019-07-03 09:30:00,1,-0.040197
3,2019-07-04 09:30:00,1,-0.023555
4,2019-07-05 09:30:00,1,-0.025090
...,...,...,...
436421,2019-12-31 09:30:00,688188,-0.035886
436422,2019-12-31 09:30:00,2960,0.012141
436423,2019-12-31 09:30:00,603115,0.031416
436424,2019-12-31 09:30:00,688321,0.018588


In [23]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainx = trainx.cuda()
trainy = trainy.cuda()
testx = testx.cuda()
testy = testy.cuda()

torch.Size([459371, 462])

In [25]:
train_dataset = Data.TensorDataset(trainx, trainy)
test_dataset = Data.TensorDataset(testx, testy)
batch_size = 512
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

test_loader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

alphanet = AlphaNet(feat_num, 30)
alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.0001
loss_function = nn.MSELoss()
optimizer = optim.RMSprop(alphanet.parameters(), lr=LR, alpha=0.9)
epoch_num = 20

for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (data, label) in enumerate(train_loader):
        data = Variable(data).float()
        label = Variable(label).float()
        pred = alphanet(data)
        print(pred.size())
        print("epoch：", epoch, "的第" "个inputs", data.data.size(), "labels", label.data.size())
        #         label = label.unsqueeze(1)
#         loss = loss_function(pred, label)
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         total_loss += loss.item()
#     total_loss = total_loss * batch_size / total_length
#     print('Epoch: ', epoch + 1, ' loss: ', total_loss)

pred_list = []
label_list = []

for _, (data, label) in enumerate(test_loader):
    data = Variable(data).float()
    pred = alphanet(data)
    pred_list.extend(pred.tolist())
    label_list.extend(label.tolist())

final = pd.concat([test_target, pd.DataFrame(pred_list)], axis=1)
alpha_name = 'AlphaNetV1_Original_Input_1208'
final.rename(columns={0: alpha_name, 'ticker': 'symbol'}, inplace=True)
final = final.reindex(columns=['symbol', 'timestamp', alpha_name,'target'])
final.set_index(['symbol', 'timestamp']).to_csv(output_path + "result/"+'%s_%s.csv' % (time_start, time_end))

  0%|          | 0/20 [00:00<?, ?it/s]

AlphaNet(
  (fc1): Linear(in_features=462, out_features=30, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (out): Linear(in_features=30, out_features=1, bias=True)
)


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [15]:
torch.cuda.device_count()

1